## DEPENDENCE INSTALL

- Dependencies install for evaluation of metrics
- Instalação das depedendencias necessárias para executar todos as avaliações

In [ ]:
%pip install numpy cython
%pip install wheel
%pip install ir_datasets
%pip install scikit-learn
%pip install nltk
%pip install requests
%pip install pandas

In [1]:
import re
import ir_datasets
import numpy as np
import requests
import json
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics import dcg_score
from sklearn.metrics import ndcg_score


In [32]:
dataset = ir_datasets.load('cranfield')


In [3]:
dataset.docs_count()

1400

## EVALUETION FUNCTIONS

- 
- Implementação de funções em algoritimos para executar as avaliações


### Calculo do DCG@k

In [72]:
def calc_dcg_k(relevance):
  result = 0
  for i in range(len(relevance)):
    log = np.log2(1 + (i + 1))
    result += (relevance[i] / log)
  return result

### Calculo NDCG@K

In [73]:
def calc_ndcg_k(relevance, true_relevance):
  return calc_dcg_k(relevance) / calc_dcg_k(true_relevance)

#### Exemplos

In [74]:
relevance = [0,0,3]
true_relevance = [4,3,3]

In [75]:
calc_ndcg_k(relevance, true_relevance)

0.20290041378171972

In [156]:
ndcg_score(np.asarray([true_relevance]), np.asarray([relevance]))

0.9412217624647558

## INDEXAR DOCUMENTOS

In [35]:
url = 'http://127.0.0.1:9200/cranfield/_doc/'

In [5]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thiagoluizrodrigues/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [55]:
count = 0
for doc in dataset.docs_iter():

  sents = list(word_tokenize(doc.text))
  # temps = []
  # for sent in sent:
  #   temps.append(re.sub('[^A-Za-z\-]', '', sent))

  mystr = ' '.join(map(str, sents))


  myobj = {
            'id': doc.doc_id,
            'title': doc.title.strip(),
            'body': mystr.strip()
          }
  response = requests.post(
      'http://127.0.0.1:9200/cranfield/_doc/' + doc.doc_id, json=myobj)
  if response.status_code == 201:
    count += 1
  else:
    print("DOC ID ", doc.doc_id, "Status ", response.status_code)
    print(response.text)

print("Indexed ", count)

DOC ID  1 Status  200
{"_index":"cranfield","_type":"_doc","_id":"1","_version":2,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":287,"_primary_term":1}
DOC ID  2 Status  200
{"_index":"cranfield","_type":"_doc","_id":"2","_version":2,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":261,"_primary_term":1}
DOC ID  3 Status  200
{"_index":"cranfield","_type":"_doc","_id":"3","_version":2,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":308,"_primary_term":1}
DOC ID  4 Status  200
{"_index":"cranfield","_type":"_doc","_id":"4","_version":2,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":279,"_primary_term":1}
DOC ID  5 Status  200
{"_index":"cranfield","_type":"_doc","_id":"5","_version":2,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":309,"_primary_term":1}
DOC ID  6 Status  200
{"_index":"cranfield","_type":"_doc","_id":"6","_version":2,"result"

In [ ]:
dataset.docs_cls()

ir_datasets.datasets.cranfield.CranfieldDoc

### Calculo do DCG@k

## EFETUAR BUSCAS

In [59]:
documents = []

In [60]:
count = 0
for query in dataset.queries_iter():

  # sent = list(word_tokenize(query.text))
  # mystr = ' '.join(map(str, sent))
  qry = "http://127.0.0.1:9200/cranfield/_doc/_search"
  x = requests.get(qry, json={
      "query": {
          "match": {
              "body": query.text
          }
      }
  })

  if x.status_code == 200:
    obj = json.loads(x.text)
    documents.append({
        "query_id":query.query_id,
        "result":obj
    })
    count += 1


print("Total..: ",count)
  # print("Query ID:", query.query_id, "Status code:", x.status_code)

Total..:  225


In [ ]:
for doc in documents[0:1]:
  print(doc) 

In [ ]:
for query in dataset.queries_iter():
    print(query)

In [223]:
import pandas as pd
pd.DataFrame(dataset.queries_iter())

,query_id,text
0,1,what similarity laws must be obeyed when const...
1,2,what are the structural and aeroelastic proble...
2,4,what problems of heat conduction in composite ...
3,8,can a criterion be developed to show empirical...
4,9,what chemical kinetic system is applicable to ...
...,...,...
220,353,papers applicable to this problem (calculation...
221,355,has anyone investigated the shear buckling of ...
222,356,papers on shear buckling of unstiffened rectan...
223,360,"in practice, how close to reality are the assu..."


In [20]:
dataset.queries_cls()._fields

('query_id', 'text')

# RELEVANCIA

In [61]:
def get_documents(query_id):
    temp_documents = []
    for doc in documents:
        if str(query_id) != doc['query_id']:
            continue
        query_results = documents[0]['result']['hits']['hits']
        for result in query_results:
            temp_documents.append(result)
    return temp_documents


In [62]:
def get_qrels(query_id):
    temp_qrels = []
    for qrel in dataset.qrels_iter():
        if qrel.query_id == str(query_id): 
            temp_qrels.append(qrel)

    return temp_qrels
    

In [63]:
def get_relevance(doc_id, qrels):
    for qrel in qrels:
        if (qrel.doc_id == str(doc_id)):
            return qrel.relevance
    return 0

In [64]:
class DocResult:
    def __init__(self, query_id, document_id, relevance, seq):
        self.query_id = query_id
        self.document_id = document_id
        self.relevance = relevance
        self.seq = seq


In [65]:
def get_query_result_docs(query_id, query_documents):
  
  temp_docs = []
  qrels = get_qrels(query_id)

  seq = 0
  for doc in query_documents:
    document_id = doc['_id']
    relevance = get_relevance(document_id, qrels)
    seq += 1
    temp_docs.append(DocResult(query_id, document_id, relevance, seq))
  
  return temp_docs

  


In [68]:

for query in dataset.queries_iter():

    result_docs = get_query_result_docs(
        query_id=query.query_id, query_documents=get_documents(query_id=query.query_id))
    i = 0

    sm = 0
    for x in result_docs:
        sm += x.relevance

    if sm > 0:

        print("++++++++++++++++++++++++")
        print("(", query.query_id, ") ", query.text)

        for x in result_docs:
            i += 1
            print("K:", i, " ", x.document_id, " ", x.relevance)


++++++++++++++++++++++++
( 1 )  what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
K: 1   184   2
K: 2   1268   0
K: 3   12   3
K: 4   486   -1
K: 5   13   4
K: 6   51   3
K: 7   14   4
K: 8   878   0
K: 9   141   0
K: 10   1361   0
++++++++++++++++++++++++
( 2 )  what are the structural and aeroelastic problems associated with flight
of high speed aircraft .
K: 1   184   2
K: 2   1268   0
K: 3   12   1
K: 4   486   -1
K: 5   13   0
K: 6   51   3
K: 7   14   4
K: 8   878   0
K: 9   141   0
K: 10   1361   0
++++++++++++++++++++++++
( 33 )  are experimental pressure distributions on bodies of revolution at angle
of attack available .
K: 1   184   0
K: 2   1268   0
K: 3   12   0
K: 4   486   0
K: 5   13   0
K: 6   51   0
K: 7   14   0
K: 8   878   0
K: 9   141   4
K: 10   1361   0
++++++++++++++++++++++++
( 56 )  what size of end plate can be safely used to simulate two-dimensional
flow conditions over a bluff cylindrical body of finit

In [95]:
query_id = 214
result_docs = get_query_result_docs(
    query_id=query_id, query_documents=get_documents(query_id=query_id))
i = 0
relevance = []
for x in result_docs[0:10]:
    i += 1
    relevance.append(x.relevance)
    #print("K:", i, " ",x.document_id, " ", x.relevance)
relevance

[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]

In [97]:
true_relevance = []
for qrel in get_qrels(query_id=query_id)[0:10]:
    true_relevance.append(qrel.relevance)
calc_ndcg_k(relevance, true_relevance)

0.11882687090889035